In [146]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import os


pd.set_option('max_rows', 500)
pd.set_option('max_colwidth',300)

os.chdir('C:\\Users\\Jason\\Introduction-to-Data-Science') #changes the directory of the script into the correct one to access the 'Energy Indicators file
os.getcwd() #shows current working directory

'C:\\Users\\Jason\\Introduction-to-Data-Science'

In [148]:
## get_list_of_university_towns
##'''Returns a DataFrame of towns and the states they are in from the 
##    university_towns.txt list. The format of the DataFrame should be:
##    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
##    columns=["State", "RegionName"]
def get_list_of_university_towns():
    new_towns = pd.read_table('university_towns.txt', header=None, names=['State', 'RegionName'])
    
    new_towns['RegionName'] = new_towns['State']
    #new_towns = new_towns.drop([200,205])
        ### dropped the two rows that weren't states but we proving difficult to format
    new_towns.loc[(new_towns['State'].str.contains("\[edit\]")),'RegionName'] = 0
        ### changed all adjacent cells in the RegionName column to 0 for easier manipulation
    new_towns['State'] = new_towns['State'].apply(lambda x: x.split('[')[0].strip())
        ### split and stripped all the [edit] away from the state names
    new_towns.loc[(new_towns['RegionName'] != 0), 'State'] = np.nan
        ### reassigned all rows w/o 0 to nan to prepare State column to forward fill
    new_towns['State'] = new_towns['State'].fillna(method='ffill')
        ### filled in all state names where there were NaN values
    new_towns = new_towns.loc[new_towns['RegionName'] != 0]
        ### removed all redundant rows with 0 and no Region information
    new_towns = new_towns.reset_index()
    new_towns = new_towns[['State','RegionName']]
    def split_func(x):
        if '(' in x:
            return x.split('(')[0].strip()
        else:
            return x.strip()
    ### function that when encounters a string with a ( splits it at that location, strips of any whitespace, if no ( does nothing
    ###'(' in new_towns['State'].iloc[0] demonstrate to myself how to correctly use the 'in' operator
    new_towns['RegionName'] = new_towns['RegionName'].apply(split_func)
    return new_towns
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
## get_recession_start()
## '''Returns the year and quarter of the recession end time as a 
## string value in a format such as 2005q3'''
## For this assignment only look at data from 2000 onward

In [141]:
def get_recession_start():
    gdp_data = pd.read_excel('gdplev.xls',header=None)
    ### read in GDP data from excel into a pandas df
    global gdp_wrk
    gdp_wrk = gdp_data.drop([0,1,2,3,5,7], axis=1) 
        ###if just indexnumbers only need to put in as integers,not strings
    col_changes = {4:'year_q', 6:'GDP_USD'}
        ### columns names to change
    gdp_wrk.rename(columns = col_changes, inplace=True)
        ### renamed columns inplace
    gdp_wrk.dropna(inplace=True)
        ### dropped all rows with NaN values
    gdp_wrk = gdp_wrk.loc[(gdp_wrk['year_q'].str.contains("20"))]
    gdp_wrk = gdp_wrk.append({'year_q':'2016q3','GDP_USD':16583.1},ignore_index=True)
        ### appends the year into the dataframe to use with downstream calcs
        ### selected only the data from 2000 and onward ###cleaned DF where can now work on it
    gdp_wrk.reset_index(drop=True,inplace=True)
        ### reset the df index, drop=True says do not try to place the OG index into the df, just reset it, if False, will place OG index as a new column
    last_num = None
    fi_num = 0
    for year, num in gdp_wrk.itertuples(index=False):
        if last_num == None:
            last_num = num
            fi_num = num
        if num < last_num < fi_num:
            year = gdp_wrk.loc[(gdp_wrk['GDP_USD'] == last_num)]
            # assigned the row watned as a dataframe to a new df object
            # select the first element of the frame, the only from the column 'year_q'
            break
        fi_num = last_num
        last_num = num
    return year.iloc[0]['year_q']


SyntaxError: 'return' outside function (<ipython-input-122-8babaf2114bd>, line 15)

In [72]:
##  get_recession_end()
##  '''Returns the year and quarter of the recession end time as a 
##  string value in a format such as 2005q3'''

In [140]:
def get_recession_end():
    get_recession_start()
    global gdp_rec
    gdp_rec = gdp_wrk.iloc[34:]
    last_num = None
    fi_num = 0
    for year, num in gdp_rec.itertuples(index=False):
        if last_num == None:
            last_num = num
            fi_num = num
        if num > last_num > fi_num:
            year = gdp_wrk.loc[(gdp_wrk['GDP_USD'] == num)]
            # assigned the row watned as a dataframe to a new df object
            # select the first element of the frame, the only from the column 'year_q'
            break
        fi_num = last_num
        last_num = num
    return year.iloc[0]['year_q']


In [74]:
## get_recession_bottom()
## Returns the year and quarter of the recession bottom time as a 
## string value in a format such as 2005q3'''



,year_q,GDP_USD
34,2008q3,14891.6
35,2008q4,14577
36,2009q1,14375
37,2009q2,14355.6
38,2009q3,14402.5
39,2009q4,14541.9
40,2010q1,14604.8
41,2010q2,14745.9
42,2010q3,14845.5
43,2010q4,14939


In [119]:
def get_recession_bottom(): 
    gdp_rec    
    gdp_min = gdp_rec['GDP_USD'].min()
     #find and assign minimum value GDP value
    rec_bot = gdp_rec['year_q'].loc[(gdp_rec['GDP_USD'] == gdp_min)]
    #query the row entry of the lowest gdp value, and select that same row from the year column, assign to rec_bot var
    return rec_bot.iloc[0]
    #output year and quarter of the corresponding lowest quarter

'2009q2'

In [11]:
### convert_housing_data_to_quarters()
#    '''Converts the housing data to quarters and returns it as mean 
#    values in a dataframe. This dataframe should be a dataframe with
#    columns for 2000q1 through 2016q3, and should have a multi-index
#    in the shape of ["State","RegionName"].
    
#    Note: Quarters are defined in the assignment description, they are
#    not arbitrary three month periods.
    
#    The resulting dataframe should have 67 columns, and 10,730 rows.

In [113]:
house_data = pd.read_csv('City_Zhvi_AllHomes.csv')

states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    # reads in all housing data
house_data.replace({'State':states},inplace=True)
    #replaces all the two-letter state names with the full spelling
house_data.set_index(['State','RegionName'],inplace=True)
    # promotes those columns passed in to a multi-level index
## REGEX THINKING AND EXP. house_data.columns.str.contains("\A(?!199)")
    #\A anchors position start expresssion at beggining ^ expresssion says, look for strings that don't have the first 3 chars as 199
    #without positional arguments regex will just try to match that a sequence does on does not have section in the string at all
    #matching your expression, have an achor where you'd like it to start
    #negative lookahead '?!' does not contain what follows i.e (?!<anything don't want to match)
house_data = house_data.loc[:,(house_data.columns.str.contains("\A(?!199)"))]
    # returns df with only columns that do not contain 199 (All of the housing data from 2000 and onward)
only_price = house_data.loc[: , house_data.columns.str.contains("20")]
    #returns the df with only price data
year_quarter = gdp_wrk.year_q.values
    #an array containing only the ordered year/quarter indices
for period, quarter in zip(np.array_split(only_price, 67, axis=1), year_quarter): #zip is impt to make list of tuples to iterate through with two iterators
    house_data[quarter] = period.mean(axis=1)                                     #otherwise python won't use both args to iterate through atthe same time
    #breaks the df into 67 3 column chunks then takes the mean across each row
    # this mean value from those 3 months is assigned to the corresponing value from the quarter they came from, and added as a new column to the housing data

#### Purely REGEX thingking and explanation now:
    #only_price = only_price.loc[:,(only_price.columns.str.contains('\d\d\d\d(?!-).+')) ]
    #matches only those entries that after 4 digits are not followed by '-' then any character 1 or more times

house_data = house_data.loc[:,(house_data.columns.str.contains('\d\d\d\d(q)\d'))]
    #this matches columns with 4 digits then a 'q' in the fifth spot followed by another digit. (Matches only the aggregated by quarter housing price data)
house_data


C:\Users\Jason\Anaconda3\lib\site-packages\ipykernel\__main__.py:29: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
Nevada,Las Vegas,132600.000000,134366.666667,135400.000000,137000.000000,139533.333333,141733.333333,143366.666667,146133.333333,149333.333333,150933.333333,...,170066.666667,173400.000000,175466.666667,177500.000000,181600.000000,186766.666667,190633.333333,194600.000000,197200.000000,199950.0
California,San Diego,222900.000000,234366.666667,245433.333333,256033.333333,267200.000000,276266.666667,284500.000000,291933.333333,301233.333333,312866.666667,...,480200.000000,489033.333333,496433.333333,503366.666667,512066.666667,519766.666667,525466.666667,529333.333333,536233.333333,539750.0
Texas,Dallas,84466.666667,83866.666667,84866.666667,87833.333333,89733.333333,89300.000000,89066.666667,90900.000000,92566.666667,93800.000000,...,106633.333333,108900.000000,111533.333333,113700.000000,121133.333333,128566.666667,134600.000000,140500.000000,144600.000000,149300.0
California,San Jose,374266.666667,406566.666667,431866.666667,455500.000000,470666.666667,470200.000000,456800.000000,445566.666667,441433.333333,457766.666667,...,679400.000000,697033.333333,714933.333333,731433.333333,756733.333333,776400.000000,789133.333333,803600.000000,818933.333333,822200.0


In [ ]:
### run_ttest():
#    '''First creates new data showing the decline or growth of housing prices
#    between the recession start and the recession bottom. Then runs a ttest
#    comparing the university town values to the non-university towns values, 
#    return whether the alternative hypothesis (that the two groups are the same)
#    is true or not as well as the p-value of the confidence. 
#    
#    Return the tuple (different, p, better) where different=True if the t-test is
#    True at a p<0.01 (we reject the null hypothesis), or different=False if 
#    otherwise (we cannot reject the null hypothesis). The variable p should
#    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
#    value for better should be either "university town" or "non-university town"
#    depending on which has a lower mean price ratio (which is equivilent to a
#    reduced market loss).'''
    

In [114]:
### first create new data showing the decline or growth of housing prices between the recession start and the rescession bottom

In [165]:
# Make another column in the DF titled 'price_ratio'
# price_ratio = quarter_before_recession/recession_bottom
# get the columns out of df that are, quarter before recession start and recession bottom for use in this calculation
recession_bottom = get_recession_bottom()
    ### assign the columns name for the recession bottom to variable
quarter_before_recession = house_data.columns[(house_data.columns.get_loc(get_recession_start()) - 1)]
    ### gets the integer index location of the recession start with get_loc method
    ### returns an integer, subtracts 1 from that to get integer index location for the quarter before
    ### then uses that integer of column loc before rec start to return the column header into the variable value
house_data['price_ratio'] = house_data[quarter_before_recession]/house_data[recession_bottom]
    ### price_ratino = 1; no change in house prices,  
    ### price_ratio < 1; an increase in value ( house prices rose during the recession)
    ### price_ratio > 1; a decrease in value (house prices over the recession)
house_data['price_ratio']
    ### then split into university towns take the mean of the price_ratio
    ### non-university towns take the mean ratio
univ_towns = get_list_of_university_towns()
univ_towns.set_index(['State','RegionName'],inplace=True)
    ### to query only the university towns I need, have to construct a list of tuples from the indexes of the univ_towns df then pass that list in
univ_towns.index.tolist()
    ### converts the indexes of this multi-level dataframe to a list of tuples that can be passed to query the matching price_ration values for the 
house_data.loc[univ_towns.index.tolist(),'price_ratio']
    ### retuns only values that are listed university towns

Alabama         Auburn                                                           NaN
                Florence                                                         NaN
                Jacksonville                                                     NaN
                Livingston                                                       NaN
                Montevallo                                                  0.950479
                Troy                                                             NaN
                Tuscaloosa                                                  1.017770
                Tuskegee                                                         NaN
Alaska          Fairbanks                                                   1.127970
Arizona         Flagstaff                                                   1.084557
                Tempe                                                       1.125462
                Tucson                                           

In [115]:
### Then runs a ttest comparing the university town values to non-university towns returns whether or not the alternative hyootheiss is true as well as the p-value of the condience
### E.C effect size of difference is one is exhibited